In [4]:
! pip install yfinance
! pip install optuna
! pip install pandas_ta

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/115.1 kB ? eta -:--:--
     -------------------------------------- 115.1/115.1 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218924 sha256=b460c03e93cf34447c0b2d272ab89c43be0516f3a6c35ebc7cb571c92c604a2b
  Stored in directory: c:\users\davide\appdata\local\pip\cache\wheels\7f\33\8b\50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas_ta



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import yfinance as yf
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
from processing.pipeline import financial_data_pipeline

C:\Users\Davide\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = yf.download("^STOXX50E", period="max")

[*********************100%%**********************]  1 of 1 completed


In [3]:
pr_data = financial_data_pipeline.fit_transform(data)

In [4]:
pr_data.head()

,Date,Open,High,Low,Close,Volume,BBL_20_2,BBM_20_2,BBU_20_2,BBB_20_2,BBP_20_2,RSI,Month_sin,Month_cos,Weekday_sin,Weekday_cos,Date_numeric,Close_t1
19,2007-04-30,4378.250000,4409.419922,4370.270020,4392.339844,0,4186.815013,4329.183008,4471.551002,6.577130,0.721808,64.894802,0.866025,-0.500000,0.000000,1.000000,13633,4415.479980
20,2007-05-02,4401.939941,4423.839844,4397.339844,4415.479980,0,4211.219223,4340.905518,4470.591812,5.975080,0.787519,67.763024,0.500000,-0.866025,0.974928,-0.222521,13635,4427.319824
21,2007-05-03,4429.500000,4432.169922,4394.189941,4427.319824,0,4238.055336,4352.794019,4467.532701,5.271956,0.824763,69.151811,0.500000,-0.866025,0.433884,-0.900969,13636,4445.589844
22,2007-05-04,4429.279785,4449.330078,4413.040039,4445.589844,0,4252.206227,4362.758521,4473.310814,5.068000,0.874625,71.212724,0.500000,-0.866025,-0.433884,-0.900969,13637,4441.310059
23,2007-05-07,4448.060059,4452.819824,4432.879883,4441.310059,0,4266.392890,4371.732520,4477.072150,4.819125,0.830253,70.032412,0.500000,-0.866025,0.000000,1.000000,13640,4411.319824


In [5]:
X = pr_data.drop('Close_t1', axis=1)
y = pr_data['Close_t1']
y = y[4:].reset_index(drop=True).values.reshape(-1, 1, 1)
X, y

(           Date         Open         High          Low        Close    Volume  \
 19   2007-04-30  4378.250000  4409.419922  4370.270020  4392.339844         0   
 20   2007-05-02  4401.939941  4423.839844  4397.339844  4415.479980         0   
 21   2007-05-03  4429.500000  4432.169922  4394.189941  4427.319824         0   
 22   2007-05-04  4429.279785  4449.330078  4413.040039  4445.589844         0   
 23   2007-05-07  4448.060059  4452.819824  4432.879883  4441.310059         0   
 ...         ...          ...          ...          ...          ...       ...   
 4290 2024-05-10  5060.729980  5096.890137  5060.729980  5085.080078  33566400   
 4291 2024-05-13  5083.540039  5087.919922  5068.060059  5078.959961  23625800   
 4292 2024-05-14  5079.370117  5083.549805  5057.939941  5080.290039  35579700   
 4293 2024-05-15  5083.250000  5102.299805  5075.060059  5100.899902  28208300   
 4294 2024-05-16  5094.770020  5101.410156  5068.109863  5072.450195  26348900   
 
          BBL_

In [6]:
window_size = 5

# Initialize an empty list to store 3D arrays
X_3d_list = []

# Iterate through each window
for i in range(len(X) - window_size + 1):
    # Extract the data for the current window
    window_data = X.iloc[i:i+window_size]
    # Convert the window data to a 2D numpy array
    window_array = window_data.drop(columns=['Date']).to_numpy()
    # Append the 2D array with the date column included as the first feature
    window_array_with_time = np.insert(window_array, 0, window_data['Date'], axis=1)
    # Append the 2D array to the list
    X_3d_list.append(window_array_with_time)

# Stack the list of 3D arrays into a single 3D numpy array
X_3d = np.stack(X_3d_list)


X_3d[1], X_3d[-1]

(array([[ 1.17806400e+18,  4.40193994e+03,  4.42383984e+03,
          4.39733984e+03,  4.41547998e+03,  0.00000000e+00,
          4.21121922e+03,  4.34090552e+03,  4.47059181e+03,
          5.97508026e+00,  7.87518674e-01,  6.77630239e+01,
          5.00000000e-01, -8.66025404e-01,  9.74927912e-01,
         -2.22520934e-01,  1.36350000e+04],
        [ 1.17815040e+18,  4.42950000e+03,  4.43216992e+03,
          4.39418994e+03,  4.42731982e+03,  0.00000000e+00,
          4.23805534e+03,  4.35279402e+03,  4.46753270e+03,
          5.27195553e+00,  8.24763210e-01,  6.91518110e+01,
          5.00000000e-01, -8.66025404e-01,  4.33883739e-01,
         -9.00968868e-01,  1.36360000e+04],
        [ 1.17823680e+18,  4.42927979e+03,  4.44933008e+03,
          4.41304004e+03,  4.44558984e+03,  0.00000000e+00,
          4.25220623e+03,  4.36275852e+03,  4.47331081e+03,
          5.06799965e+00,  8.74625079e-01,  7.12127240e+01,
          5.00000000e-01, -8.66025404e-01, -4.33883739e-01,
         -9.

In [7]:
X_train, X_remainder, y_train, y_remainder = train_test_split(X_3d, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_remainder, y_remainder, test_size=0.333, shuffle=False)
X_test, X_forecast, y_test, y_forecast = train_test_split(X_test, y_test, test_size=0.2, shuffle=False)

In [8]:
len(X_train), len(X_val), len(X_test), len(X_forecast), len(y_val)

(2990, 855, 341, 86, 855)

In [9]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [10]:
batch_size = 32

In [11]:
train_dataset = CustomDataset(X_train, y_train)
validation_dataset = CustomDataset(X_val, y_val)
test_dataset = CustomDataset(X_test, y_test)
forecast_dataset = CustomDataset(X_forecast, y_forecast)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
forecast_dataloader = DataLoader(forecast_dataset, batch_size=batch_size, shuffle=False)

In [21]:
X_train[0]

array([[ 1.17789120e+18,  4.37825000e+03,  4.40941992e+03,
         4.37027002e+03,  4.39233984e+03,  0.00000000e+00,
         4.18681501e+03,  4.32918301e+03,  4.47155100e+03,
         6.57712988e+00,  7.21808406e-01,  6.48948025e+01,
         8.66025404e-01, -5.00000000e-01,  0.00000000e+00,
         1.00000000e+00,  1.36330000e+04],
       [ 1.17806400e+18,  4.40193994e+03,  4.42383984e+03,
         4.39733984e+03,  4.41547998e+03,  0.00000000e+00,
         4.21121922e+03,  4.34090552e+03,  4.47059181e+03,
         5.97508026e+00,  7.87518674e-01,  6.77630239e+01,
         5.00000000e-01, -8.66025404e-01,  9.74927912e-01,
        -2.22520934e-01,  1.36350000e+04],
       [ 1.17815040e+18,  4.42950000e+03,  4.43216992e+03,
         4.39418994e+03,  4.42731982e+03,  0.00000000e+00,
         4.23805534e+03,  4.35279402e+03,  4.46753270e+03,
         5.27195553e+00,  8.24763210e-01,  6.91518110e+01,
         5.00000000e-01, -8.66025404e-01,  4.33883739e-01,
        -9.00968868e-01,  1.3

In [12]:
for batch in train_dataloader:
    pass

In [13]:
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout_prob):
        super(GRUModel, self).__init__()

        self.layer_dim = num_layers
        self.hidden_dim = hidden_dim

        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initializing hidden state for first input with zeros
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()

        # Forward propagation by passing in the input and hidden state into the model
        out, _ = self.gru(x, h0.detach())

        # Reshaping the outputs in the shape of (batch_size, seq_length, hidden_size)
        # so that it can fit into the fully connected layer
        out = out[:, -1, :]

        # Convert the final state to our desired output shape (batch_size, output_dim)
        out = self.fc(out)
        return out

In [23]:
input_dim = X_train.shape[2]
output_dim = 1

In [24]:
num_epochs = 10
num_trials = 10

In [25]:
# Define the objective function for hyperparameter optimization
def objective(trial):
    # Sample hyperparameters
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True)
    hidden_dim = trial.suggest_int('hidden_dim', 32, 512, log=True)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout = trial.suggest_float('dropout', 0.0, 0.5)

    # Define and initialize the model
    model = GRUModel(input_dim, hidden_dim, num_layers, output_dim, dropout)

    # Define optimizer and criterion
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_dataloader:
            optimizer.zero_grad()
            inputs = inputs.float()
            targets = targets.float()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        validation_loss = 0
        with torch.no_grad():
            for inputs, targets in validation_dataloader:
                inputs = inputs.float()
                targets = targets.float()
                outputs = model(inputs)
                validation_loss += criterion(outputs, targets).item()

        validation_loss /= len(validation_dataloader)

        # Report intermediate results to Optuna
        trial.report(validation_loss, epoch)

        # Handle pruning based on the intermediate results
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return validation_loss

# Create a study object with TPE sampler
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=num_trials)

# Get the best hyperparameters
best_params = study.best_params




[I 2024-05-19 12:07:46,036] A new study created in memory with name: no-name-dd5d2abc-0348-40a2-ba37-339256fe3b8c
C:\Users\Davide\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Davide\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([14, 1, 1])) that is different to the input size (torch.Size([14, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Davide\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size(

KeyboardInterrupt: 

In [229]:
best_params

{'learning_rate': 0.0037751854616517506,
 'hidden_dim': 291,
 'num_layers': 2,
 'dropout': 0.08608821446980097}

In [238]:
best_lr = best_params['learning_rate']
best_hd = best_params['hidden_dim']
best_nl = best_params['num_layers']
best_do = best_params['dropout']

In [261]:
X_tv = np.concatenate((X_train, X_val), axis=0)
y_tv = np.concatenate((y_train, y_val), axis=0)

tv_dataset = CustomDataset(X_tv, y_tv)
tv_dataloader = DataLoader(tv_dataset, batch_size=batch_size, shuffle=False)

In [263]:
cv_model = GRUModel(input_dim, best_hd, best_nl, output_dim, best_do)

In [264]:
cv_num_epochs = 10
cv_criterion = nn.MSELoss()
cv_optimizer = torch.optim.Adam(cv_model.parameters(), lr=best_lr)

In [265]:
# Training loop
for epoch in range(cv_num_epochs):
    cv_model.train()
    for inputs, targets in tv_dataloader:
        cv_optimizer.zero_grad()
        inputs = inputs.float()
        targets = targets.float()
        outputs = cv_model(inputs)
        loss = cv_criterion(outputs, targets)
        loss.backward()
        cv_optimizer.step()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([23, 1, 1])) that is different to the input size (torch.Size([23, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [266]:
cv_model.eval()
with torch.no_grad():
    test_loss = 0
    for inputs, targets in test_dataloader:
        inputs = inputs.float()
        targets = targets.float()
        outputs = cv_model(inputs)
        test_loss += cv_criterion(outputs, targets).item()

    test_loss /= len(test_dataloader)

print(f'Test Loss: {test_loss}')

Test Loss: 8714271.818181818


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([23, 1, 1])) that is different to the input size (torch.Size([23, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [267]:
cv_model.eval()
with torch.no_grad():
    forecast_loss = 0
    for inputs, targets in forecast_dataloader:
        inputs = inputs.float()
        targets = targets.float()
        outputs = cv_model(inputs)
        forecast_loss += cv_criterion(outputs, targets).item()

    forecast_loss /= len(forecast_dataloader)

print(f'Forecast Loss: {forecast_loss}')

Forecast Loss: 13555126.666666666


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 1])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([22, 1, 1])) that is different to the input size (torch.Size([22, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
